## Tabela Hash - colisão por encadeamento

Uma tabela Hash é uma estrutura de dados de acesso direto, pois o índice do dado a ser acessado é conhecido.

O índice de acesso a cada valor é dado por uma função hash `h(k) -> i`, que recebe a chave K e retorna o índice para acessar o valor V. 

Este notebook apresenta a implementação de uma tabela Hash com tratamento de colisões, utilizando o **método de encadeamento**, que consiste na criação de uma lista para armazenar os valores com colisão.

A tabela será usada para armazenar nomes próprios (V) e a chave (K) será a primeira letra de cada palavra.

O notebook https://github.com/Marcosddf/algoritmoseestruturasdedados/blob/master/tabela_hash_simples.ipynb possui uma tabela hash sem tratamento de colisões.

In [1]:
#include <iostream>
#include <stdlib.h> 
#include <string.h>
using namespace std;

#define M 19; //tamanho da tabela Hash
#define A_ASCII 65;

Definição da lista que será usada para armazenar os elementos da tabela Hash. A inclusão dos elementos com colisão é feita de forma sequencial, sem ordenação.

In [2]:
struct elemento {
    char chave; //a chave é opcional, poderia ser calculada usando a 1a letra do valor.
    char valor[10];
    elemento *prox;
}

Busca elementos na lista.

In [3]:
elemento *buscaLista (elemento *in, char chave) {
    elemento *p = in;
    if (p == NULL) return p;
    while (p->prox != NULL) {
        if (p->chave == chave)
            return p;
        p=p->prox;
    }
    return p;
}

Inicia um elemento da lista.

In [4]:
elemento *inicia (char chave, const char *valor){
    elemento *p = (elemento *)malloc (sizeof (elemento));
    p->chave = chave;
    strcpy(p->valor, valor);
    p->prox = NULL;
    return p;
}

Imprime todos os elementos da lista.

In [5]:
void imprimeLista (elemento *p) {
  if (p == NULL) return;
  cout << p->valor << ":";
  return imprimeLista (p->prox);
}

Imprime o valor específico de um elemento da lista.

In [6]:
void imprimeValor (elemento *p) {
  if (p != NULL)
      cout << p->valor;
}

Insere um novo elemento na lista.

In [7]:
elemento *insere (elemento *p, const char *valor) {
   elemento *novo = inicia (valor[0], valor);
   if (p == NULL) return novo;
   while (p->prox != NULL)
       p=p->prox;
   p->prox = novo;   
   return p;
}

Exclui um elemento da lista.

In [8]:
elemento *excluiLista(elemento *p, char chave) {
    elemento *anterior, *primeiro = p;
    if (p->chave == chave) {
        anterior = p->prox;
        free(p);
        return anterior;
    }
    while (p != NULL) {
        if (p->chave == chave){
            anterior->prox = p->prox;
            free(p);
            return primeiro;
        }
        anterior = p;
        p = p->prox;
    }
    return primeiro;
}

O tamanho da tabela Hash depende do número de chaves existentes e da capacidade de armazenamento. 

In [9]:
int tamHash(){
    return M;
}

O **método da divisão** é bastante usado para implementar funções hash.

Dada uma tabela de tamanho **M**, a função hash será `h(k) -> k mod m`. M deverá ser preferencialmente um número primo, e potencias de 2 devem ser evitadas. 

Nesta implementação o M escolhido é 19.

Esta função pode ser trocada para implementar diferentes métodos.

In [10]:
int hash_divisao(int k){
    cout << "\nk mod m " << (k % tamHash()) << ' ';
    return (k % tamHash());
}

Função para iniciar todos os elementos da tabela hash.

A tabela apontará para uma lista de elementos.

In [11]:
void inicia(elemento tabelaHash[]){
    for (int i = 0; i < tamHash(); i ++){
        tabelaHash[i].chave = '\0';
        tabelaHash[i].valor[0] = '\0';
        tabelaHash[i].prox = NULL;
    }
}

Função que converte um caracter ASCII para um índice numérico, que será a chave da string. Leva em consideração letras maiúsculas do alfabeto.

In [12]:
int char_to_chave(char chr){
    char c = toupper(chr);
    return c - A_ASCII;
}

Imprime os elementos da tabela Hash, isto é, uma lista e seus elementos para cada posição da tabela.

In [13]:
void imprime(elemento tabela[]){
    cout << endl << endl;
    for (int i = 0; i < tamHash(); i ++) {
        cout << i << ":{";
        cout << tabela[i].valor << ".";
        if (tabela[i].prox != NULL)
            imprimeLista(tabela[i].prox);
        cout << "}.";
    }
}

Inclui um elemento na tabela Hash. Os elementos com colisão serão incluídos em uma lista encadeada, de acordo com a ordem de inclusão, sem ordenação. 

Cada elemento deve ter uma chave única, porém mais de uma chave pode __mapear__ para o mesmo índice.

In [14]:
void inclui (elemento tabelaHash[], const char *valor) {
    int i = hash_divisao(char_to_chave(valor[0]));
    if (tabelaHash[i].chave == '\0'){
        tabelaHash[i].chave = toupper(valor[0]);
        strcpy(tabelaHash[i].valor, valor);
    }
    else  //possui colisoes, incluir na lista
        insere(&tabelaHash[i],valor);
}

Busca um elemento na tabela Hash. Calcula o valor do índice usando a função Hash, e busca o elemento na lista de colisões. Pode ser uma string vazia.

In [15]:
elemento *busca (elemento tabelaHash[], char chave){
    return buscaLista(& tabelaHash[hash_divisao(char_to_chave(chave))], chave);
}

Função de exclusão na tabela hash. Procura a posição baseada na chave, e exclui o primeiro elemento, ou então procura na lista encadeada.

Há um tratamento diferente para o primeiro elemento porque todos os elementos da tabela hash são previamente alocados.

In [16]:
void exclui (elemento tabelaHash[], char chave){
    int indice = hash_divisao(char_to_chave(chave));
    elemento *aux;
    if (tabelaHash[indice].chave == chave){
        if (tabelaHash[indice].prox == NULL){
            tabelaHash[indice].chave = '\0';
            tabelaHash[indice].valor[0] = '\0';
            tabelaHash[indice].prox = NULL;
        } else {
            aux = tabelaHash[indice].prox;
            tabelaHash[indice].chave = tabelaHash[indice].prox->chave;
            strcpy(tabelaHash[indice].valor, tabelaHash[indice].prox->valor);
            tabelaHash[indice].prox = tabelaHash[indice].prox->prox;
            free(aux);
        }
    }
    else 
        excluiLista (& tabelaHash[indice], chave);        
}

Inclui, busca e exclui elementos desta tabela Hash, com tratamento de colisões usando o **método de encadeamento**.

In [17]:
void iniciaprograma(){
    
    int t = tamHash();
    elemento *e;
    elemento tabelaHash[19];
    inicia(tabelaHash);
    
    inclui(tabelaHash, "Ariel");
    inclui(tabelaHash, "Lucas");
    inclui(tabelaHash, "Rafael");
    inclui(tabelaHash, "Pamela");
    inclui(tabelaHash, "Tamara");
    inclui(tabelaHash, "Val");
    inclui(tabelaHash, "Claus");
    
    imprime(tabelaHash);
    
    cout << endl;
    
    imprimeValor ( busca(tabelaHash, 'R') );
    imprimeValor ( busca(tabelaHash, 'P') );
    imprimeValor ( busca(tabelaHash, 'K') );
    imprimeValor ( busca(tabelaHash, 'V') );
    imprimeValor ( busca(tabelaHash, 'C') );
    
    exclui(tabelaHash,'L');
    exclui(tabelaHash,'V');
    
    imprime(tabelaHash);
    
    imprimeValor (busca(tabelaHash, 'R'));
    imprimeValor (busca(tabelaHash, 'P'));
    imprimeValor (busca(tabelaHash, 'K'));    
    
}

In [18]:
iniciaprograma();


k mod m 0 
k mod m 11 
k mod m 17 
k mod m 15 
k mod m 0 
k mod m 2 
k mod m 2 

0:{Ariel.Tamara:}.1:{.}.2:{Val.Claus:}.3:{.}.4:{.}.5:{.}.6:{.}.7:{.}.8:{.}.9:{.}.10:{.}.11:{Lucas.}.12:{.}.13:{.}.14:{.}.15:{Pamela.}.16:{.}.17:{Rafael.}.18:{.}.

k mod m 17 Rafael
k mod m 15 Pamela
k mod m 10 
k mod m 2 Val
k mod m 2 Claus
k mod m 11 
k mod m 2 

0:{Ariel.Tamara:}.1:{.}.2:{Claus.}.3:{.}.4:{.}.5:{.}.6:{.}.7:{.}.8:{.}.9:{.}.10:{.}.11:{.}.12:{.}.13:{.}.14:{.}.15:{Pamela.}.16:{.}.17:{Rafael.}.18:{.}.
k mod m 17 Rafael
k mod m 15 Pamela
k mod m 10 